In [ ]:
import yfinance as yf
from datetime import datetime

from pyspark.sql import SparkSession
import pyspark.pandas as ps

In [ ]:
list_symbol=['ACWI','QQQ','SPY','ETH-USD']
start_param="2022-07-01"
end_param="2022-07-30"
#end_param=end= datetime.now().strftime("%Y-%m-%d")


In [ ]:
spark=SparkSession.builder.appName("FundFeeding_Yahoo").getOrCreate()
#spark

ps.set_option('compute.ops_on_diff_frames', True)
#spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

In [ ]:
input_cols=['Ticker','Date','Open','High','Low','Close','Volume']
list_not_found=[]
df=ps.DataFrame(columns=input_cols)
for ticker in list_symbol:

   try:
    print(ticker)

    df_pd = yf.download(ticker, start=start_param, end=end_param)
    dfx=  ps.from_pandas(df_pd) 
    
    if (dfx.empty==False) or (dfx is not None):
        dfx['Ticker'] = ticker
        dfx=dfx.reset_index()
        dfx=dfx[input_cols]
        df=ps.concat([df,dfx])
        #print(dfx.tail())
        
    else:
        print(f"{ticker} No data found for this date range, symbol may be delisted")
        list_not_found.append(ticker) 


   except Exception as ex:
      print(str(ex))
      list_not_found.append(ticker)  


ACWI
[*********************100%***********************]  1 of 1 completed
QQQ
[*********************100%***********************]  1 of 1 completed
SPY
[*********************100%***********************]  1 of 1 completed
ETH-USD
[*********************100%***********************]  1 of 1 completed


In [ ]:
df.head()

,Ticker,Date,Open,High,Low,Close,Volume
0,ACWI,2022-07-01,83.500000,84.550003,82.930000,84.480003,3230800
1,ACWI,2022-07-05,82.949997,84.070000,82.400002,84.050003,5108900
2,ACWI,2022-07-06,83.989998,84.589996,83.419998,84.169998,3530200
3,ACWI,2022-07-07,84.699997,85.650002,84.699997,85.510002,3197700
4,ACWI,2022-07-08,85.129997,85.900002,84.820000,85.449997,2727300


In [ ]:
df_corr=ps.DataFrame()
for symbol in list_symbol:
  df_by_symbol=ps.sql("select Close from {df} where Ticker={symbol}")
  df_by_symbol=df_by_symbol.rename(columns={'Close':symbol})   
  #print(df_by_symbol.head())
  df_corr=ps.concat([df_corr, df_by_symbol], axis=1)
  
  
df_corr=df_corr.dropna()
corrMatrix = df_corr.corr()

print(corrMatrix)
    

             ACWI       QQQ       SPY   ETH-USD
ACWI     1.000000  0.937565  0.989717  0.546418
QQQ      0.937565  1.000000  0.971098  0.517412
SPY      0.989717  0.971098  1.000000  0.569826
ETH-USD  0.546418  0.517412  0.569826  1.000000


In [ ]:
df_corr

,ACWI,QQQ,SPY,ETH-USD
0,84.480003,282.130005,381.239990,1059.767334
1,84.050003,286.959991,381.959991,1066.512817
2,84.169998,288.799988,383.250000,1073.766968
3,85.510002,294.980011,388.989990,1151.059082
4,85.449997,295.350006,388.670013,1134.541016
